In [ ]:
import json, datetime, pickle
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fontsize=9
result_path = "../Results"
TM_path = "../TM"
Fig_path = "../Fig"

## Adapt the paths to point to the directory containing the results

In [ ]:
def load_data(path, scales, show=False,change_eval=False, topo={},surf=False, print_result=False):
    dataset = {}
    results = []

    for i in scales:
        with open(f"{path}/scale_{i}.txt", "r") as f:
            dataset[f"scale_{i}"] = []
            lines = f.readlines()
            for line in lines:
                line = line.split(",")

                dataset[f"scale_{i}"].append((line[0], json.loads(','.join(line[1:]))))
        dataset[f"scale_{i}"] = sorted(dataset[f"scale_{i}"], key=lambda x: x[1]["config"]["timestep"])

    for key in dataset.keys():
        
        result_dict = {"no_sleep": [], "ol": [], "ol_1": [], "ol_next": [], "ol_next_1": [], "ts": [], "actual_sleep": [], "max_bw": [], "overload": [], "sleep_edges": [], "reroute_budget": []}
        for i, element in enumerate(dataset[key]):

            if "bundle" not in element[1]["config"].keys() or element[1]["config"]["bundle"] == False:
                pass
            else:
                pass

            if len(element[1]["ol_next"]["ol_active"])-len(element[1]["ol_next"]["ol_sleep"]) > 0 or len(element[1]["ol_next"]["ol_active_1"])-len(element[1]["ol_next"]["ol_sleep_1"]) > 0:
                print(element[1]["config"]["timestep"])
                continue
            
            if surf:
                # timesteps with lots of missing data
                #if element[1]["config"]["timestep"] in [1712283300,1712368800,1713039900]:
                #    continue
                pass
            ol_now = element[1]["result"]["ol_now"]
            ol_next = element[1]["result"]["ol_next"]

            result_dict["ts"].append(datetime.datetime.fromtimestamp((element[1]["config"]["timestep"])))
            result_dict["no_sleep"].append(element[1]["result"]["no_sleep"])
            result_dict["reroute_budget"].append(element[1]["config"]["reroute_budget"])

            link_speeds = {}
            for link in element[1]["sleep_edges"]:
                if (link[0],link[1],tuple(link[2])) not in topo.edges:
                    continue
                speed = topo[link[0]][link[1]][tuple(link[2])]["max_bw"]
                if speed not in link_speeds:
                    link_speeds[speed] = 1
                else:
                    link_speeds[speed] += 1
            result_dict["max_bw"].append(link_speeds)
            ol_sum, ol_sum_active = 0, 0
            for ol_link in element[1]["ol_next"]["ol_sleep_1"]:
                ol_sum += ol_link[0]-ol_link[1]
            for ol_link in element[1]["ol_next"]["ol_active_1"]:
                ol_sum_active += ol_link[0]-ol_link[1]
            
            if change_eval:
                result_dict["sleep_edges"].append([ (x[0], x[1], tuple(x[2])) if x[0] < x[1] else (x[1], x[0], tuple(reversed(x[2]))) for x in element[1]["sleep_edges"]])
            result_dict["overload"].append((ol_sum,ol_sum_active))
            result_dict["ol"].append(ol_now[0])
            
            result_dict["ol_1"].append(ol_now[1])
            result_dict["ol_next"].append(ol_next[0])
            result_dict["ol_next_1"].append(ol_next[1])
            if ol_now == [0,0] and ol_next == [0,0]:
                result_dict["actual_sleep"].append(element[1]["result"]["no_sleep"])
            else:
                result_dict["actual_sleep"].append(0)


        link_speeds = {}
        for bw in result_dict["max_bw"]:
            for speed in sorted(bw.keys()):
                if speed not in link_speeds:
                    link_speeds[speed] = bw[speed]/len(result_dict["max_bw"])
                else:
                    link_speeds[speed] += bw[speed]/len(result_dict["max_bw"])

        if change_eval:
            change = []
            for element1, element2 in zip(result_dict["sleep_edges"],result_dict["sleep_edges"][1:]):
                change_1 = len(set(element1).difference(set(element2)))
                change_2 = len(set(element2).difference(set(element1)))
                change.append((change_1+change_2)/element[1]["result"]["no_sleep"])
            del result_dict["sleep_edges"]
        
        if show or print_result:
            print(key)
            print(link_speeds)
            print(f'no of timesteps: {len(result_dict["no_sleep"])}')
            print(f'links turned off: {np.mean(result_dict["no_sleep"])}, actual after correction: {np.mean(result_dict["actual_sleep"])}')
            print(f'Avg. link over 80%: {np.mean(result_dict["ol"])}, no. of times it happens: {np.count_nonzero(result_dict["ol"])}')
            print(f'Avg. link over 100%: {np.mean(result_dict["ol_1"])}, no. of times it happens: {np.count_nonzero(result_dict["ol_1"])}')
            print(f'Avg. link over 80% next 5 minutes: {np.mean(result_dict["ol_next"])}, {np.count_nonzero(result_dict["ol_next"])}')
            print(f'Avg. link over 100% next 5 minutes: {np.mean(result_dict["ol_next_1"])}, {np.count_nonzero(result_dict["ol_next_1"])}')
            print(f'Avg. reroute bugdet: {np.mean(result_dict["reroute_budget"])}')

            if show:
                fig = px.line(x=result_dict["ts"],y=result_dict["no_sleep"],title=key)
                fig.show()
                fig2 = px.line(x = result_dict["ts"],y=[result_dict["ol"],result_dict["ol_1"], result_dict["ol_next"], result_dict["ol_next_1"]],title=key)
                fig2.show()

                if change_eval:
                    fig3 = px.line(x = result_dict["ts"][1:],y=change,title=key).update_layout(yaxis_title="Sleep Link Changes",xaxis_title="Time")
                    fig3.show()
         
        if change_eval:
            results.append((result_dict, link_speeds, change))
        else:
            results.append((result_dict, link_speeds))
    return results

In [ ]:
path = f"{result_path}/switch"
G, _ = pickle.load(open(f"{TM_path}/Switch/scale_1/{1703858100+0*300}_1.pkl", "rb"))
scale_switch = [1,2,5,10]
results = load_data(path, scale_switch, show=False, change_eval=True, topo=G,print_result=True)

In [ ]:
path = f"{result_path}/surf"
G_surf, _ = pickle.load(open(f"{TM_path}/Surfnet/scale_1/{'2024-04-01T00:05:00Z_1'}.pkl", "rb"))
scale_surf = [1,2,5,10]
results_surf = load_data(path, scale_surf, show=False, change_eval=True, topo=G_surf, surf=True, print_result=True)

## Create the plots

In [ ]:
def create_link_speed(results, G, name=""):
    link_speeds_sleep = results[0][1]
    link_speeds = {}

    for link in G.edges:
        speed = G[link[0]][link[1]][link[2]]["max_bw"]
        if speed not in link_speeds:
            link_speeds[speed] = 1/2
        else:
            link_speeds[speed] += 1/2

    for key in link_speeds.keys():
        if key not in link_speeds_sleep.keys():
            link_speeds_sleep[key] = 0
        link_speeds[key] = link_speeds[key] - link_speeds_sleep[key]

    speed_df = {"Link capacity": list(link_speeds_sleep.keys()) + list(link_speeds.keys()), "no_links": list(link_speeds_sleep.values()) + list(link_speeds.values()), "type": ["Sleeping links"]*len(link_speeds) + ["All links"]*len(link_speeds_sleep)}
    speed_df = pd.DataFrame(speed_df)

    fig = px.bar(speed_df.sort_values("type",ascending=False), x="Link capacity", y="no_links", color="type", color_discrete_sequence=px.colors.qualitative.G10)
    fig.update_xaxes(title=dict(text="Link capacity [Gbit/s]",font=dict(size=fontsize)), type='category',categoryorder='array', categoryarray=sorted(speed_df["Link capacity"]))
    ytitle = {"x":0, "y":1.2, "xref":"paper", "yref":"paper", "text":'Number of links', "showarrow":False, "xanchor":'left'}
    custom_layout = {"title":None, "font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations":[ytitle]}
    fig.update_layout(custom_layout)
    fig.update_layout(yaxis_title=None)
    if name == "surf":
        fig.update_layout(yaxis=dict(range=[0,650],dtick=150))
        fig.data[1].update(showlegend=False)
        fig.update_layout(legend=dict(orientation="v",yanchor="top",y=2, yref="paper", xanchor="left",x=0))
        fig.update_layout(barmode="stack", legend_title="", showlegend=True, width=150, height=140, margin=dict(l=5, r=5, t=40, b=25))
    else:
        fig.data[0].update(showlegend=False)
        fig.update_layout(yaxis=dict(range=[0,180],dtick=40))
        fig.update_layout(legend=dict(orientation="v",yanchor="top",y=2, yref="paper", xanchor="left",x=0))
        fig.update_layout(barmode="stack", legend_title="",legend={'traceorder':'reversed'}, width=150, height=140, margin=dict(l=5, r=5, t=20, b=10))
    fig.show()
    fig.write_image(f"{Fig_path}/speed_{name}.pdf", scale=1)
    return fig

In [ ]:
def create_timeseries(results,name=""):
    result_dict = results[0][0]
    fig = go.Figure()
    fig.add_trace(px.line(x=result_dict["ts"],y=result_dict["no_sleep"], color_discrete_sequence=px.colors.qualitative.G10, render_mode="svg").data[0])
    fig.data[0].update(line=dict(width=0.6))
    annotations = []
    no_1 = 2.38
    no_2 = 7.44 
    if name == "surf":
        for i in range(35,39,1):
            annotations.append({"x":1, "y":i*no_2, "xref":"x domain", "yref":"y", "text":f'{i}', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
            fig.add_hline(y=i*no_2, line_dash="dot", line_color='rgb(100,100,100)', line_width=0.5, layer='below')
    else:
        for i in range(32,39,2):
            annotations.append({"x":1, "y":i*no_1, "xref":"x domain", "yref":"y", "text":f'{i}', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
            fig.add_hline(y=i*no_1, line_dash="dot", line_color='rgb(100,100,100)', line_width=0.5, layer='below')
        
    fig.update_xaxes(title=None)
    annotations.append({"x":0, "y":1.2, "xref":"paper", "yref":"paper", "text":'Number of links turned off', "showarrow":False, "xanchor":'left'})
    annotations.append({"x":1, "y":1.2, "xref":"x domain", "yref":"y domain", "text":'% of links', "showarrow":False, "xanchor":'right', "font": dict(size=fontsize)})
    custom_layout = {"title":None, "font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations":annotations}
    fig.update_layout(custom_layout)
    fig.update_layout(yaxis_title=None)
    fig.update_layout(legend_title="", width=320, height=100, margin=dict(l=20, r=20, t=20, b=10))
    fig.show()
    fig.write_image(f"{Fig_path}/timeseries_{name}.pdf", scale=1)
    return fig

In [ ]:
def create_stability(results, name=""):
    result_dict, _, change = results[0]
    change = [x*100 for x in change]
    if name == "surf":
        window = 200
    else:
        window = 1000
    local_avg = []
    for i in range(len(change)):
        if i < window:
            local_avg.append(np.mean(change[:i+1]))
        else:
            local_avg.append(np.mean(change[i-window:i+1]))

    stability_df = pd.DataFrame({"Time": result_dict["ts"][1:], "Link changes": change, "Moving average": local_avg})
    fig_stab = px.line(stability_df, x="Time", y=["Link changes","Moving average"], color_discrete_sequence=px.colors.qualitative.G10, render_mode="svg")
    fig_stab.data[0].update(line=dict(width=0.6))
    fig_stab.update_xaxes(title=None)
    ytitle = {"x":0, "y":1.18, "xref":"paper", "yref":"paper", "text":'Link changes in %', "showarrow":False, "xanchor":'left'}
    custom_layout = {"title":None, "font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations":[ytitle]}
    fig_stab.update_layout(custom_layout)
    fig_stab.update_layout(yaxis_title=None)
    fig_stab.update_layout(legend=dict(orientation="h",yanchor="top",y=1.6, yref="paper", xanchor="left",x=0,entrywidth=100))
    fig_stab.update_layout(legend_title="", width=320, height=125, margin=dict(l=0, r=0, t=10, b=20))
    fig_stab.show()
    fig_stab.write_image(f"{Fig_path}/stability_{name}.pdf", scale=1)
    return fig_stab

In [ ]:
def create_mistakes(results, scales, name=""):
    
    ol_results = {"Current load > 80%": [], "Current load > 100%": [], "Next load > 80%": [], "Next load > 100%": [], "Reroute Budget": []}
    ol_results["scale"] = scales
    for i, result in enumerate(results):
        result_dict, link_speeds_sleep, change = result
        ol_results["Current load > 80%"].append(np.count_nonzero(result_dict["ol"]))
        ol_results["Current load > 100%"].append(np.count_nonzero(result_dict["ol_1"]))
        ol_results["Next load > 80%"].append(np.count_nonzero(result_dict["ol_next"]))
        ol_results["Next load > 100%"].append(np.count_nonzero(result_dict["ol_next_1"]))
        ol_results["Reroute Budget"].append(np.mean(result_dict["reroute_budget"]))
    ol_results = pd.DataFrame(ol_results)

    colors = px.colors.qualitative.G10
    fig = make_subplots(rows=2, shared_xaxes=True, vertical_spacing=0.2)
    for i in [2,3,0,1]:
        fig.add_trace(px.line(ol_results, x="scale", y=["Current load > 80%","Current load > 100%","Next load > 80%","Next load > 100%"],markers=True, color_discrete_sequence=[colors[0], colors[1], colors[0], colors[1]]).data[i], col=1, row=1)
        fig.data[-1]["legendgroup"] = 1
    fig.add_trace(px.line(ol_results, x="scale", y=["Reroute Budget"],markers=True, color_discrete_sequence=["black"]).data[0], col=1, row=2)
    fig.data[4]["legendgroup"] = 2
    fig.update_xaxes(title=dict(text="Scale", font=dict(size=fontsize)),tickvals=[1,2,5,10],title_standoff = 4, col=1, row=2)
    if name == "surf":
        text="Mistakes over 14 days"
    else:
        text="Mistakes over 75 days"
    ytitle = {"x":0, "y":1.3, "xref":"x domain", "yref":"y domain", "text": text, "showarrow":False, "xanchor":'left'}
    ytitle2 = {"x":0, "y":1.34, "xref":"x2 domain", "yref":"y2 domain", "text": "Reroute Budget [Gbps]", "showarrow":False, "xanchor":'left'}
    custom_layout = {"title":None, "font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations":[ytitle, ytitle2]}
    fig['data'][0].update(line=dict(dash="dot"), marker=dict(symbol="square"))
    fig['data'][1].update(line=dict(dash="dot"), marker=dict(symbol="diamond"))
    fig["data"][2].update(marker=dict(symbol="square"))
    fig['data'][3].update(marker=dict(symbol="diamond"))
    fig['data'][4].update(showlegend=False)
    for i in range(5):
        fig['data'][i].update(marker=dict(size=6))
    fig.update_layout(custom_layout)
    fig.update_layout(yaxis_title=None, legend_tracegroupgap=45)
    if name == "surf":
        fig['data'][0].update(showlegend=False)
        fig['data'][2].update(showlegend=False)
        fig.update_layout(legend=dict(orientation="v",yanchor="top",y=1.6, yref="paper", xanchor="left",x=0))
        fig.update_layout(legend_title=None, showlegend=True, legend_traceorder="reversed", width=150, height=200, margin=dict(l=5, r=15, t=5, b=10))
    else:
        fig['data'][1].update(showlegend=False)
        fig['data'][3].update(showlegend=False)
        fig.update_layout(legend=dict(orientation="v",yanchor="top", y=1.6, yref="paper", xanchor="left",x=0))
        fig.update_layout(legend_title=None, showlegend=True, legend_traceorder="reversed", width=150, height=200, margin=dict(l=5, r=15, t=5, b=10))
    fig.update_layout(legend=dict(itemsizing="trace", itemwidth=30))
    fig.show()
    fig.write_image(f"{Fig_path}/mistakes_{name}.pdf", scale=1)
    return fig

In [ ]:
def create_savings(results, scales, G, name=""):
    ol_results = {"Links asleep": [], "% of links asleep": []}
    ol_results["scale"] = scales
    for i, result in enumerate(results):
        result_dict, link_speeds_sleep, change = result
        ol_results["Links asleep"].append(np.mean(result_dict["actual_sleep"]))
        ol_results["% of links asleep"].append(np.mean(result_dict["actual_sleep"])/(len(G.edges)/2) * 100)
    ol_results = pd.DataFrame(ol_results)

    colors = px.colors.qualitative.G10
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(px.line(ol_results, x="scale", y="Links asleep", markers=True, color_discrete_sequence=px.colors.qualitative.G10).data[0], secondary_y=False)
    fig['data'][0].update(marker=dict(size=6))
    
    annotations=[]
    annotations.append({"x":0, "y":1.2, "xref":"x domain", "yref":"y domain", "text":'Links asleep', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
    annotations.append({"x":1, "y":1.2, "xref":"x domain", "yref":"y domain", "text":'% of links', "showarrow":False, "xanchor":'right', "font": dict(size=fontsize)})

    no_1 = 2.38
    no_2 = 7.44

    if name == "surf":
        for i in range(34,39,2):
            annotations.append({"x":1, "y":i*no_2, "xref":"x domain", "yref":"y", "text":f'{i}', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
            fig.add_hline(y=i*no_2, line_dash="dot", line_color='rgb(100,100,100)', line_width=0.5, layer='below')
    else:
        for i in range(28,39,4):
            annotations.append({"x":1, "y":i*no_1, "xref":"x domain", "yref":"y", "text":f'{i}', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
            fig.add_hline(y=i*no_1, line_dash="dot", line_color='rgb(100,100,100)', line_width=0.5, layer='below')

    fig.update_xaxes(title=dict(text="Scale", font=dict(size=fontsize)), tickvals=scales)
    
    custom_layout = {"font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations":annotations}
    fig.update_layout(custom_layout)
    if name == "surf":
        fig.update_layout(yaxis=dict(range=[243,287]))
    else:
        fig.update_layout(yaxis=dict(range=[63,88], dtick=5))

    fig.update_layout(yaxis_title=None)
    if name == "surf":
        fig.update_layout(legend_title="", width=150, height=125, margin=dict(l=0, r=6, t=20, b=20))
    else:
        fig.update_layout(legend_title="", width=150, height=125, margin=dict(l=0, r=6, t=20, b=20))
    fig.show()
    fig.write_image(f"{Fig_path}/savings_{name}.pdf", scale=1)
    return fig

In [ ]:
speed_surf = create_link_speed(results_surf, G_surf, "surf")
create_timeseries(results_surf, "surf")
create_stability(results_surf, "surf")
create_mistakes(results_surf, scale_surf, "surf")
saving_surf = create_savings(results_surf, scale_surf, G_surf, "surf")

In [ ]:
speed = create_link_speed(results, G, "switch")
create_timeseries(results, "switch")
create_stability(results, "switch")
create_mistakes(results, scale_switch, "switch")
saving = create_savings(results, scale_switch, G, "switch")

In [ ]:
results_surf_budget = []
for budget in [0.01,0.1,1,10,100]:
    G_surf, _ = pickle.load(open(f"{TM_path}/Surfnet/scale_1/2024-04-01T00:05:00Z_1.pkl", "rb"))
    if budget==1:
        path = f"{result_path}/surf"
    else:
        path = f"{result_path}/surf_budget_{budget}"

    surf_scale = [1]
    result_dict = load_data(path, surf_scale, show=False, change_eval=False, topo=G_surf, surf=True)[0][0]
    mistakes = np.count_nonzero(result_dict["ol"]) + np.count_nonzero(result_dict["ol_1"]) + np.count_nonzero(result_dict["ol_next"]) + np.count_nonzero(result_dict["ol_next_1"]) 
    results_surf_budget.append((np.mean(result_dict["actual_sleep"]),mistakes))
print(results_surf_budget)

In [ ]:
results_switch_budget = []
for budget in [0.01,0.1,1,10,100]:
    G, _ = pickle.load(open(f"{TM_path}/Switch/scale_1/{1703858100}_1.pkl", "rb"))
    if budget==1:
        path = f"{result_path}/switch"
    else:
        path = f"{result_path}/switch_budget_{budget}"

    scale_switch = [1]
    result_dict = load_data(path, scale_switch, show=False, change_eval=False, topo=G)[0][0]
    mistakes = np.count_nonzero(result_dict["ol"]) + np.count_nonzero(result_dict["ol_1"]) + np.count_nonzero(result_dict["ol_next"]) + np.count_nonzero(result_dict["ol_next_1"]) 
    results_switch_budget.append((np.mean(result_dict["actual_sleep"]),mistakes))
print(results_switch_budget)

In [ ]:
def create_budget(results_budget,name=""):
    color = px.colors.qualitative.G10
    fig = make_subplots(rows=1, cols=1,  specs=[[{"secondary_y": True}]])
    #fig.add_trace(px.line(x=[0.01,0.1,1,10,100],y=[x[0] for x in results_surf_budget],title="Average links asleep").data[0])
    budget = {"scale": [0.01,0.1,1,10,100], "Links asleep": [x[0] for x in results_budget], "Mistakes": [x[1] for x in results_budget]}
    budget = pd.DataFrame(budget)
    fig.add_trace(px.line(budget, x="scale",y=["Links asleep"], color_discrete_sequence=[color[0]], markers=True).data[0], row=1,col=1)
    fig.add_trace(px.line(budget, x="scale",y=["Mistakes"], color_discrete_sequence=[color[1]], markers=True).data[0], secondary_y=True, row=1,col=1)
    fig.update_xaxes(title=dict(text="Reroute Scale",font=dict(size=fontsize)), type='category',categoryorder='array',ticks="outside",tickvals=[0.01,0.1,1,10,100], range=[-0.2,4.2])
    fig.data[1].update(line=dict(dash="dot"))
    for i in range(2):
        fig['data'][i].update(marker=dict(size=6))
    y=1.14
    annotations=[]
    annotations.append({"x":0, "y":y, "xref":"x domain", "yref":"y domain", "text":'Links asleep', "showarrow":False, "xanchor":'left', "font": dict(size=fontsize)})
    annotations.append({"x":1, "y":y, "xref":"x domain", "yref":"y domain", "text":'Mistakes', "showarrow":False, "xanchor":'right', "font": dict(size=fontsize)})

    custom_layout = {"title":None, "font":{"size": fontsize, "color": 'rgb(0,0,0)'},"yaxis":{'title':{'text':''}},"annotations": annotations}
    fig.update_layout(custom_layout)
    fig.update_layout(yaxis_title=None)
    fig.update_yaxes(tickmode="sync")
    if name == "surf":
        fig.data[0].update(showlegend=False)
        fig.update_layout(yaxis=dict(range=[258,292]), yaxis2=dict(range=[-0.2,3.2]))
        fig.update_layout(legend=dict(orientation="v",yanchor="bottom", y=1.15, yref="paper", xanchor="left",x=0))
        fig.update_layout(legend_title="", showlegend=True, width=160, height=200, margin=dict(l=5, r=5, t=20, b=50))
    else:
        fig.data[1].update(showlegend=False)
        fig.update_layout(yaxis=dict(range=[68,92]),yaxis2=dict(range=[-1200,13200]))
        fig.update_layout(legend=dict(orientation="v",yanchor="bottom", y=1.15, yref="paper", xanchor="left",x=0))
        fig.update_layout(legend_title="", showlegend=True, width=160, height=200, margin=dict(l=5, r=5, t=20, b=50))
    fig.show()
    fig.write_image(f"{Fig_path}/budget_{name}.pdf", scale=1)


In [ ]:
create_budget(results_surf_budget,"surf")
create_budget(results_switch_budget,"switch")

In [ ]:
path = f"{result_path}/switch_2"
G, _ = pickle.load(open(f"{TM_path}/Switch/scale_1/{1703858100}_1.pkl", "rb"))

_ = load_data(path, [1], show=False, change_eval=True, topo=G, print_result=True)

In [ ]:
path = f"{result_path}/surf_2"
G_surf, _ = pickle.load(open(f"{TM_path}/Surfnet/scale_1/{'2024-04-01T00:05:00Z_1'}.pkl", "rb"))

_ = load_data(path, [1], show=False, change_eval=True, topo=G_surf, surf=True, print_result=True)